In [1]:
%load_ext autoreload
%autoreload 2


import mahotas as mh
import numpy as np
import os
import time

from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
%matplotlib inline

import mlproof as mlp

mlproof/tifffile.py:1995: UserWarning: failed to import _tifffile.decodepackbits
  warnings.warn("failed to import %s" % module_function)
mlproof/tifffile.py:1995: UserWarning: failed to import _tifffile.decodelzw
  warnings.warn("failed to import %s" % module_function)
mlproof/tifffile.py:1995: UserWarning: failed to import _tifffile.unpackints
  warnings.warn("failed to import %s" % module_function)


In [2]:
PATCH_PATH = '/home/d/patches/cylinder_small1_rgba/'

In [136]:


def generate_patches(start_slice, end_slice):

    patch_index = 0

    all_patches_count = 0
    patch_list = []
    all_error_patches = []
    all_correct_patches = []
    
    for z in range(start_slice, end_slice):

        t0 = time.time()
        print 'working on slice', z
        input_image, input_prob, input_mask, input_gold, input_rhoana = mlp.Util.read_section('/home/d/data/cylinder/',z)

        error_patches, patches = mlp.Patch.patchify_maxoverlap(input_image, input_prob, input_mask, input_rhoana, input_gold, sample_rate=1)

#         for e in patches:
#             if e['image'].max() == 0.:
#                 print 'ZERRRRO', z
        
        print 'Generated', len(error_patches), 'split error patches and', len(patches), ' correct patches in', time.time()-t0, 'seconds.'
        
        patch_list.append(patches)
        all_error_patches += error_patches
        all_correct_patches += patches
        
    
    
    NO_PATCHES = len(all_error_patches) + len(all_correct_patches)

    print 'We have a total of',NO_PATCHES,'patches.'
    print 'Errors:',len(all_error_patches)
    print 'Correct:',len(all_correct_patches)    
    
    PATCH_BYTES = 75*75
    P_SIZE = (NO_PATCHES, 4, 75,75) # rather than raveled right now
    
    p_rgba = np.zeros(P_SIZE, dtype=np.float32)
    p_rgba_large = np.zeros(P_SIZE, dtype=np.float32)    
    
#     p_image = np.zeros(P_SIZE, dtype=np.float32)
#     p_prob = np.zeros(P_SIZE, dtype=np.float32)
#     p_binary = np.zeros(P_SIZE, dtype=np.bool)
#     p_merged_array = np.zeros(P_SIZE, dtype=np.bool)
# #     p_dyn_obj = np.zeros((NO_PATCHES, PATCH_BYTES),dtype=np.bool)
# #     p_dyn_bnd = np.zeros((NO_PATCHES, PATCH_BYTES),dtype=np.bool)
#     p_border_overlap = np.zeros(P_SIZE, dtype=np.bool)
#     p_larger_border_overlap = np.zeros(P_SIZE, dtype=np.bool)    
    p_target = np.zeros(NO_PATCHES)


    i = 0
    for p in all_error_patches:
#         print 'testing one y=1', p['image'].max()
        p_rgba[i][0] = p['image']
        p_rgba[i][1] = p['prob'] 
        p_rgba[i][2] = p['merged_array']
        p_rgba[i][3] = p['border_overlap']
        
        p_rgba_large[i][0] = p['image']
        p_rgba_large[i][1] = p['prob']    
        p_rgba_large[i][2] = p['merged_array']
        p_rgba_large[i][3] = p['larger_border_overlap']        
        
#         p_image[i] = (p['image'].astype(np.float32)/255.)#.ravel()
#         p_prob[i] = (p['prob'].astype(np.float32)/255.)#.ravel()
#         p_binary[i] = p['binary']#.ravel()
#         p_merged_array[i] = p['merged_array']#.ravel()
# #         p_dyn_obj[i] = p['dyn_obj'].ravel()
# #         p_dyn_bnd[i] = p['dyn_bnd'].ravel()
#         p_border_overlap[i] = p['border_overlap']#.ravel()
#         p_larger_border_overlap[i] = p['larger_border_overlap']#.ravel()
        p_target[i] = 1 # <--- important
        i += 1

        
    for p in all_correct_patches:
        print 'testing one y=0', p['image'].max()
        p_rgba[i][0] = p['image']
        p_rgba[i][1] = p['prob']    
        p_rgba[i][2] = p['merged_array']
        p_rgba[i][3] = p['border_overlap']
        
        p_rgba_large[i][0] = p['image']
        p_rgba_large[i][1] = p['prob']       
        p_rgba_large[i][2] = p['merged_array']
        p_rgba_large[i][3] = p['larger_border_overlap']        
        
#         p_image[i] = (p['image'].astype(np.float32)/255.)#.ravel()
#         p_prob[i] = (p['prob'].astype(np.float32)/255.)#.ravel()
#         p_binary[i] = p['binary']#.ravel()
#         p_merged_array[i] = p['merged_array']#.ravel()
# #         p_dyn_obj[i] = p['dyn_obj'].ravel()
# #         p_dyn_bnd[i] = p['dyn_bnd'].ravel()
#         p_border_overlap[i] = p['border_overlap']#.ravel()
#         p_larger_border_overlap[i] = p['larger_border_overlap']#.ravel()
        p_target[i] = 0 # <--- important
        i+=1
        
    
    return p_rgba, p_rgba_large, p_target



def shuffle_in_unison_inplace(a, b, c):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p], c[p]



def run(start_slice, end_slice, filename):
    
    if not os.path.exists(PATCH_PATH):
        os.makedirs(PATCH_PATH)
    
    p = generate_patches(start_slice, end_slice)
    
    shuffled = shuffle_in_unison_inplace(p[0],
                                         p[1],
                                         p[2]
                                        )
    
    print 'saving..'
    np.savez(PATCH_PATH+filename+'.npz', rgba=shuffled[0],
                                         rgba_large=shuffled[1])
    np.savez(PATCH_PATH+filename+'_targets.npz', targets=shuffled[2])
    print 'Done!'
    
    return shuffled


In [154]:
sh1[0][0][0]

array([[ 0.32549021,  0.28235295,  0.29019609, ...,  0.65882355,
         0.67843139,  0.72156864],
       [ 0.42352942,  0.36470589,  0.41960785, ...,  0.60000002,
         0.71764708,  0.74901962],
       [ 0.43529412,  0.46666667,  0.48235294, ...,  0.60392159,
         0.6901961 ,  0.70980394],
       ..., 
       [ 0.65490198,  0.61176473,  0.56470591, ...,  0.52941179,
         0.50980395,  0.48235294],
       [ 0.60784316,  0.60000002,  0.58823532, ...,  0.51764709,
         0.52549022,  0.52156866],
       [ 0.64705884,  0.6156863 ,  0.6156863 , ...,  0.52941179,
         0.54509807,  0.52156866]], dtype=float32)

In [137]:
# sh1 = run(40,50,'training')
sh1 = run(40,42,'train')
# sh2 = run(60,62,'test')

working on slice 40
Loading z00000040.png
Generated 1132 split error patches and 1132  correct patches in 338.530647039 seconds.
working on slice 41
Loading z00000041.png
Generated 1220 split error patches and 1220  correct patches in 355.197210073 seconds.
We have a total of 4704 patches.
Errors: 2352
Correct: 2352
testing one y=0 0.839216
testing one y=0 0.839216
testing one y=0 0.839216
testing one y=0 0.839216
testing one y=0 0.8
testing one y=0 0.8
testing one y=0 0.807843
testing one y=0 0.807843
testing one y=0 0.8
testing one y=0 0.8
testing one y=0 0.847059
testing one y=0 0.847059
testing one y=0 0.811765
testing one y=0 0.811765
testing one y=0 0.847059
testing one y=0 0.847059
testing one y=0 0.784314
testing one y=0 0.784314
testing one y=0 0.8
testing one y=0 0.8
testing one y=0 0.8
testing one y=0 0.8
testing one y=0 0.835294
testing one y=0 0.835294
testing one y=0 0.8
testing one y=0 0.8
testing one y=0 0.835294
testing one y=0 0.835294
testing one y=0 0.803922
testing

In [156]:
X_train, y_train, X_test, y_test = mlp.Patch.load('cylinder_small1_rgba')

Loaded /home/d/patches//cylinder_small1_rgba/ in 0.000553131103516 seconds.


In [139]:
X_train['rgba'].shape

(24296, 4, 75, 75)

In [157]:
X_train['rgba'][0][0].max()

0.81568629

In [133]:
X_train['rgba_large'][0][0].max()

0.0

In [134]:
X_train['rgba'][1][1].max()

0.97254902

In [ ]:
X_train['rgba'][0][1].max()

In [50]:
X_train['rgba'][2][3].max()

1.0

In [131]:
y_train[2]

0

In [130]:
y_train[0]

0